# cDCGAN (conditional Deep Convolutional GAN)

**Author:** Teebone Ding

**Date:** 2019 Feb

Based on DCGAN (Radford, Metz, 2016) [Paper Link](https://arxiv.org/pdf/1511.06434.pdf)

Some useful refs while I implementing my cDCGAN:
* keras GAN [Link](https://github.com/eriklindernoren/Keras-GAN/blob/master/cgan/cgan.py)
* cDCGAN in tensorflow version [Link](https://github.com/znxlwm/tensorflow-MNIST-cGAN-cDCGAN)
* cDCGAN using keras [Link](https://medium.com/@utk.is.here/training-a-conditional-dc-gan-on-cifar-10-fce88395d610)

Using **MNIST dataset** as playground.
My SW/HW setting:
* ASUS nVidia GTX 1060 3GB
* RAM: 16GB
* CPU: Intel Core i5-7400 3.00GHz
* Ubuntu 16.04 LTS

In [1]:
import tensorflow as tf
from tensorflow import keras

import numpy as np
import matplotlib.pyplot as plt

In [2]:
print(tf.__version__)

1.12.0


## Load MNIST data and data explore

In [3]:
# Load MNIST fashion data
#mnist_dataset = keras.datasets.fashion_mnist
mnist_dataset = keras.datasets.mnist
(x_train, y_train), (_, _) = mnist_dataset.load_data()

In [4]:
print(x_train.shape)
print(y_train.shape)

(60000, 28, 28)
(60000,)


In [5]:
from tensorflow.keras.utils import to_categorical
y_train_one_hot = to_categorical(y_train)

In [6]:
print(y_train_one_hot.shape)
print(y_train_one_hot[:10])

(60000, 10)
[[0. 0. 0. 0. 0. 1. 0. 0. 0. 0.]
 [1. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 1. 0. 0. 0. 0. 0.]
 [0. 1. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 1.]
 [0. 0. 1. 0. 0. 0. 0. 0. 0. 0.]
 [0. 1. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 1. 0. 0. 0. 0. 0. 0.]
 [0. 1. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 1. 0. 0. 0. 0. 0.]]


In [7]:
y_train[:10]

array([5, 0, 4, 1, 9, 2, 1, 3, 1, 4], dtype=uint8)

# Preprocess training data

In [9]:
x_train = (x_train.astype(np.float32) - 127.5) / 127.5
#x_train = x_train.astype(np.float32)/255.0

In [10]:
x_train = np.expand_dims(x_train,axis=3) # add color channel
#x_train = x_train.reshape((60000,784))

In [11]:
x_train.shape

(60000, 28, 28, 1)

## Build Generator and Discriminator
Try CNN layers to build GAN (cDCGAN).

In [12]:
from tensorflow.keras.layers import Input, Dense, Concatenate, Reshape, Flatten, Conv2D, Conv2DTranspose
from tensorflow.keras.layers import BatchNormalization, LeakyReLU, ReLU, UpSampling2D, Dropout
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam

In [13]:
z = Input(shape=(100,))    # Uniform distribution noise input
x = Input(shape=(28,28,1)) # image, by real dataset or generator
#x = Input(shape=(784,))
y = Input(shape=(10,))     # label, MNIST one hot encoded with 10 labels

In [15]:
def build_generator(z,y):
    layer = Concatenate()([z,y])
    
    layer = Dense((4*4*256))(layer)
    layer = BatchNormalization(momentum=0.8)(layer)
    layer = LeakyReLU(alpha=0.2)(layer)
    layer = Reshape((4,4,256))(layer)
    
    layer = Conv2DTranspose(128,5,strides=2,padding='same')(layer)
    layer = BatchNormalization(momentum=0.8)(layer)
    layer = LeakyReLU(alpha=0.2)(layer)

    layer = Conv2DTranspose(64,5,strides=3, activation='relu',padding='same')(layer)
    layer = BatchNormalization(momentum=0.8)(layer)
    layer = LeakyReLU(alpha=0.2)(layer)

    layer = Conv2DTranspose(1,5,strides=1, activation='tanh',padding='valid')(layer)
    #layer = Flatten()(layer)
    
    model = Model([z,y],layer)

    return model,layer

G,G_out = build_generator(z,y)
G.summary() # not compiled, combined with discriminator and compile.

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 100)          0                                            
__________________________________________________________________________________________________
input_3 (InputLayer)            (None, 10)           0                                            
__________________________________________________________________________________________________
concatenate (Concatenate)       (None, 110)          0           input_1[0][0]                    
                                                                 input_3[0][0]                    
__________________________________________________________________________________________________
dense (Dense)                   (None, 4096)         454656      concatenate[0][0]                
__________

In [17]:
def build_discriminator(x,y):
    layer = Conv2D(32,3,strides=2,padding="same")(x)
    layer = LeakyReLU(alpha=0.2)(layer)
    layer = Dropout(0.25)(layer)
    
    layer = Conv2D(64,3,strides=2,padding="same")(layer)
    layer = BatchNormalization(momentum=0.8)(layer)
    layer = LeakyReLU(alpha=0.2)(layer)
    layer = Dropout(0.25)(layer)
    
    layer = Conv2D(128,3,strides=2,padding="same")(layer)
    layer = BatchNormalization(momentum=0.8)(layer)
    layer = LeakyReLU(alpha=0.2)(layer)
    layer = Dropout(0.25)(layer)
    
    layer = Flatten()(layer)
    layer = Concatenate()([layer, y])
    layer = Dense(256,activation='relu')(layer)
    
    layer = Dense(1,activation='sigmoid')(layer)
    
    model = Model([x,y],layer)
    return model, layer

D, D_out = build_discriminator(x,y)
#D.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_2 (InputLayer)            (None, 28, 28, 1)    0                                            
__________________________________________________________________________________________________
conv2d (Conv2D)                 (None, 14, 14, 32)   320         input_2[0][0]                    
__________________________________________________________________________________________________
leaky_re_lu_3 (LeakyReLU)       (None, 14, 14, 32)   0           conv2d[0][0]                     
__________________________________________________________________________________________________
dropout (Dropout)               (None, 14, 14, 32)   0           leaky_re_lu_3[0][0]              
__________________________________________________________________________________________________
conv2d_1 (

In [18]:
# While training....
# 1. train discriminator first
# 2. train a combined (G+D) model, with D is not trainable (only train G)
# 3. calculate loss value and save weights

In [19]:
# Compile Discriminator model
#sgd = SGD(lr=0.1, momentum=0.5 ,decay= 1.00004)
adam = Adam(0.0002, 0.5)
D.compile(optimizer=adam, loss='binary_crossentropy', metrics=['accuracy'])
D.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_2 (InputLayer)            (None, 28, 28, 1)    0                                            
__________________________________________________________________________________________________
conv2d (Conv2D)                 (None, 14, 14, 32)   320         input_2[0][0]                    
__________________________________________________________________________________________________
leaky_re_lu_3 (LeakyReLU)       (None, 14, 14, 32)   0           conv2d[0][0]                     
__________________________________________________________________________________________________
dropout (Dropout)               (None, 14, 14, 32)   0           leaky_re_lu_3[0][0]              
__________________________________________________________________________________________________
conv2d_1 (

In [22]:
# Compile combined G+D model with D is not trainable
img = G([z,y])
D.trainable = False
valid = D([img,y])
C = Model([z,y], valid)
C.summary()
C.compile(optimizer=adam,loss='binary_crossentropy')

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 100)          0                                            
__________________________________________________________________________________________________
input_3 (InputLayer)            (None, 10)           0                                            
__________________________________________________________________________________________________
model (Model)                   (None, 28, 28, 1)    1497601     input_1[0][0]                    
                                                                 input_3[0][0]                    
__________________________________________________________________________________________________
model_1 (Model)                 (None, 1)            620801      model[2][0]                      
          

# Train conditional GAN

In [23]:
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3' 
tf.logging.set_verbosity(tf.logging.ERROR)

In [24]:
# dataset: x_train, y_train_one_hot

In [25]:
# Randomly generate images from Generator network
def sample_gen_imgs(epoch):
    gen_labels = np.repeat(np.arange(10),10)
    gen_labels_cat = to_categorical(gen_labels, num_classes=10)
    noises = np.random.normal(0,1,(100,100))
    gen_imgs = G.predict([noises,gen_labels_cat])
    gen_imgs = 0.5 * gen_imgs + 0.5

    r,c = 10,10
    fig, axs = plt.subplots(r, c, figsize=(20, 20))
    cnt = 0
    for i in range(r):
        for j in range(c):
            axs[i,j].imshow(gen_imgs[cnt,:,:,0], cmap='gray')
            #axs[i,j].set_title("Cat: %s" % y_cat[gen_labels[cnt]])
            axs[i,j].set_title("Digits: %s" % gen_labels[cnt])
            axs[i,j].axis('off')
            cnt += 1
    fig.savefig("images/gen_mnist_%d.png"%epoch)
    plt.close()

In [28]:
import pdb
import datetime
def train(epochs = 10 ,batch_size = 128): 
    real_label = np.ones((batch_size,1))
    fake_label = np.zeros((batch_size,1))
    
    for epoch in range(epochs):
        epoch += 1
        for step in range(int(60000/batch_size)):
            # Train Discriminator once
            # sample real data from training dataset
            idx = np.random.randint(0, x_train.shape[0], batch_size)
            imgs, labels = x_train[idx], y_train_one_hot[idx]
            # sample noises from normal dist.
            # Somehow I tried uniform dist but cannot trained very well.
            noises = np.random.normal(0,1,(batch_size,100))
            # generate fake images
            gen_imgs = G.predict([noises,labels])
            #pdb.set_trace()
            # Train Discriminator
            d_real_loss = D.train_on_batch([imgs,labels], real_label) 
            d_fake_loss = D.train_on_batch([gen_imgs,labels],fake_label)
            d_loss = 0.5*np.add(d_real_loss,d_fake_loss)
           
            # Train Combined model (generator)
            gen_labels = to_categorical(np.random.randint(0,10, batch_size),num_classes=10)
            # train generator
            g_loss = C.train_on_batch([noises,gen_labels], real_label)
            print("[%s] D-Loss value: %.5f Acc: %.5f G-Loss value: %.5f in epoch: %d"%(datetime.datetime.now(),d_loss[0],d_loss[1], g_loss, epoch),end='\r')
        
        if epoch % 5 == 0:
            print("[%s] D-Loss value: %.5f Acc: %.5f G-Loss value: %.5f in epoch: %d"%(datetime.datetime.now(),d_loss[0],d_loss[1], g_loss, epoch))
            sample_gen_imgs(epoch)
            G.save("models/G_mnist_%d.h5"%epoch)
            C.save("models/C_mnist_%d.h5"%epoch)
            D.save("models/D_mnist_%d.h5"%epoch)

In [30]:
train(epochs=20, batch_size = 128)

[2019-02-17 00:27:45.613657] D-Loss value: 0.65597 Acc: 0.58594 G-Loss value: 0.81672 in epoch: 5
[2019-02-17 00:31:04.388116] D-Loss value: 0.65510 Acc: 0.57031 G-Loss value: 0.92401 in epoch: 10
[2019-02-17 00:34:20.722020] D-Loss value: 0.62054 Acc: 0.65625 G-Loss value: 0.88760 in epoch: 15
[2019-02-17 00:37:38.796056] D-Loss value: 0.63429 Acc: 0.64062 G-Loss value: 0.88756 in epoch: 20


## Generated images after 20 epochs
![alt text](images/gen_mnist_fashion_20.png "Gen image 200 epochs")
